In [2]:
##细分进行整理预测，最后再整合
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')


flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

temp = trans[trans.columns[1:5]].drop_duplicates()
new_trans = pd.DataFrame()
for dt in trans['date_dt'].drop_duplicates().values:
    temp['date_dt'] = dt
    new_trans = pd.concat([new_trans,temp],axis=0,ignore_index=True)
new_trans = pd.merge(new_trans,trans,how='left',on=new_trans.columns.tolist()).fillna(0)
new_trans = new_trans[trans.columns]
trans = new_trans.copy()
del new_trans
print(flow.columns)
print(trans.columns)

cc_se = pd.Series(trans['o_city_code'].value_counts().index)
dc_se = pd.Series(trans['o_district_code'].value_counts().index)

trans['o_city_code'] = trans['o_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['o_district_code'] = trans['o_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

trans['d_city_code'] = trans['d_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['d_district_code'] = trans['d_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['cnt']
for label in labels:
    train = []
    test = []
    for va in trans[['o_city_code','o_district_code','d_city_code','d_district_code']].drop_duplicates().values:
        occ = va[0]
        odc = va[1]
        dcc = va[2]
        ddc = va[3]
        price = trans[(trans['o_city_code']==occ)&(trans['o_district_code']==odc)&(trans['d_city_code']==dcc)&(trans['d_district_code']==ddc)][label].values.tolist()
        k = len(price)
        diff = 30
        l = [occ,odc,dcc,ddc]
        l.extend(price[k-diff:k])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [occ,odc,dcc,ddc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=VBE)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='cnt':
        sub = test_x[[0,1,2,3]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['o_city_code'] = sub[0].map(cc_se)
sub['o_district_code'] = sub[1].map(dc_se)
sub['d_city_code'] = sub[2].map(cc_se)
sub['d_district_code'] = sub[3].map(dc_se)

res = pd.read_csv('result/submission_30.csv',header=None)  # 1329
res.columns = flow.columns.tolist()
res['flow_out'] = res['district_code'].map(sub.groupby('o_district_code')['cnt'].sum())
res['flow_in'] = res['district_code'].map(sub.groupby('d_district_code')['cnt'].sum())
res.to_csv('result/submission_part_30.csv',header=None,index=False)

0.1316


Index(['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in',
       'flow_out'],
      dtype='object')
Index(['date_dt', 'o_city_code', 'o_district_code', 'd_city_code',
       'd_district_code', 'cnt'],
      dtype='object')
(2186380, 34)
(9506, 34)
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[617]	train's rmse: 2.2892	evals's rmse: 3.1454
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[488]	train's rmse: 2.8258	evals's rmse: 3.35921
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[423]	train's rmse: 2.47652	evals's rmse: 3.39921
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[336]	train's rmse: 2.47056	evals's rmse: 3.3392
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[627]	train's rmse: 2.56894	evals's rmse: 3.04209
[3.145

In [10]:
##细分进行整理预测，最后再整合
import pandas as pd
import numpy as np
import os
import time
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score,mean_squared_error
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')


flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

temp = trans[trans.columns[1:5]].drop_duplicates()
new_trans = pd.DataFrame()
for dt in trans['date_dt'].drop_duplicates().values:
    temp['date_dt'] = dt
    new_trans = pd.concat([new_trans,temp],axis=0,ignore_index=True)
new_trans = pd.merge(new_trans,trans,how='left',on=new_trans.columns.tolist()).fillna(0)
new_trans = new_trans[trans.columns]
trans = new_trans.copy()
del new_trans
print(flow.columns)
print(trans.columns)

cc_se = pd.Series(trans['o_city_code'].value_counts().index)
dc_se = pd.Series(trans['o_district_code'].value_counts().index)

trans['o_city_code'] = trans['o_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['o_district_code'] = trans['o_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

trans['d_city_code'] = trans['d_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['d_district_code'] = trans['d_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['cnt']
for label in labels:
    train = []
    test = []
    for va in trans[['o_city_code','o_district_code','d_city_code','d_district_code']].drop_duplicates().values:
        occ = va[0]
        odc = va[1]
        dcc = va[2]
        ddc = va[3]
        price = trans[(trans['o_city_code']==occ)&(trans['o_district_code']==odc)&(trans['d_city_code']==dcc)&(trans['d_district_code']==ddc)][label].values.tolist()
        k = len(price)
        diff = 30
        l = [occ,odc,dcc,ddc]
        l.extend(price[k-diff:k])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [occ,odc,dcc,ddc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])
    
    
    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        col = train_x.columns[4:]
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            model = LassoCV()
            model.fit(train_x[col].loc[train_part_index],train_y.loc[train_part_index])
            evals_ypre = model.predict(train_x[col].loc[evals_index])
            best_score = np.sqrt(mean_squared_error(train_y.loc[evals_index].values,evals_ypre))
            num+=1
            valid_ypre = model.predict(test_x[col])
            df['pred_'+str(num)]=valid_ypre
            rmse.append(best_score)
            print(best_score)
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='cnt':
        sub = test_x[[0,1,2,3]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['o_city_code'] = sub[0].map(cc_se)
sub['o_district_code'] = sub[1].map(dc_se)
sub['d_city_code'] = sub[2].map(cc_se)
sub['d_district_code'] = sub[3].map(dc_se)

res = pd.read_csv('result/submission_30.csv',header=None)  # 1329
res.columns = flow.columns.tolist()
res['flow_out'] = res['district_code'].map(sub.groupby('o_district_code')['cnt'].sum())
res['flow_in'] = res['district_code'].map(sub.groupby('d_district_code')['cnt'].sum())
res.to_csv('result/submission_part_30_Lasso.csv',header=None,index=False)
1311

Index(['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in',
       'flow_out'],
      dtype='object')
Index(['date_dt', 'o_city_code', 'o_district_code', 'd_city_code',
       'd_district_code', 'cnt'],
      dtype='object')
(2186380, 34)
(9506, 34)
3.709023718659636
3.5802069983257905
3.6421341805328566
3.5936766041384987
3.432282681232046
[3.709023718659636, 3.5802069983257905, 3.6421341805328566, 3.5936766041384987, 3.432282681232046]
3.591464836577765


In [13]:
##细分进行整理预测，最后再整合
import pandas as pd
import numpy as np
import os
import time
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score,mean_squared_error
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')


flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')
trans = trans[trans['date_dt']>=20170901].reset_index(drop='True')


temp = trans[trans.columns[1:5]].drop_duplicates()
new_trans = pd.DataFrame()
for dt in trans['date_dt'].drop_duplicates().values:
    temp['date_dt'] = dt
    new_trans = pd.concat([new_trans,temp],axis=0,ignore_index=True)
new_trans = pd.merge(new_trans,trans,how='left',on=new_trans.columns.tolist()).fillna(0)
new_trans = new_trans[trans.columns]
trans = new_trans.copy()
del new_trans
print(flow.columns)
print(trans.columns)

cc_se = pd.Series(trans['o_city_code'].value_counts().index)
dc_se = pd.Series(trans['o_district_code'].value_counts().index)

trans['o_city_code'] = trans['o_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['o_district_code'] = trans['o_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

trans['d_city_code'] = trans['d_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['d_district_code'] = trans['d_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['cnt']
for label in labels:
    train = []
    test = []
    for va in trans[['o_city_code','o_district_code','d_city_code','d_district_code']].drop_duplicates().values:
        occ = va[0]
        odc = va[1]
        dcc = va[2]
        ddc = va[3]
        price = trans[(trans['o_city_code']==occ)&(trans['o_district_code']==odc)&(trans['d_city_code']==dcc)&(trans['d_district_code']==ddc)][label].values.tolist()
        k = len(price)
        diff = 30
        l = [occ,odc,dcc,ddc]
        l.extend(price[k-diff:k])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [occ,odc,dcc,ddc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])
    
    
    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        col = train_x.columns[4:]
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            model = LassoCV()
            model.fit(train_x[col].loc[train_part_index],train_y.loc[train_part_index])
            evals_ypre = model.predict(train_x[col].loc[evals_index])
            best_score = np.sqrt(mean_squared_error(train_y.loc[evals_index].values,evals_ypre))
            num+=1
            valid_ypre = model.predict(test_x[col])
            df['pred_'+str(num)]=valid_ypre
            rmse.append(best_score)
            print(best_score)
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='cnt':
        sub = test_x[[0,1,2,3]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['o_city_code'] = sub[0].map(cc_se)
sub['o_district_code'] = sub[1].map(dc_se)
sub['d_city_code'] = sub[2].map(cc_se)
sub['d_district_code'] = sub[3].map(dc_se)

res = pd.read_csv('result/submission_30.csv',header=None)  # 1329
res.columns = flow.columns.tolist()
res['flow_out'] = res['district_code'].map(sub.groupby('o_district_code')['cnt'].sum())
res['flow_in'] = res['district_code'].map(sub.groupby('d_district_code')['cnt'].sum())
res.to_csv('result/submission_part_30_Lasso_3.csv',header=None,index=False)
1338

Index(['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in',
       'flow_out'],
      dtype='object')
Index(['date_dt', 'o_city_code', 'o_district_code', 'd_city_code',
       'd_district_code', 'cnt'],
      dtype='object')
(1311828, 34)
(9506, 34)
2.5884471344726947
2.71439360021903
2.632753954754065
2.783767740113759
2.638086009526294
[2.5884471344726947, 2.71439360021903, 2.632753954754065, 2.783767740113759, 2.638086009526294]
2.671489687817169


In [2]:
##细分进行整理预测，最后再整合
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')


flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')
print(trans.shape)
trans = trans[trans['date_dt']>=20171001].reset_index(drop=True)
print(trans.shape)
temp = trans[trans.columns[1:5]].drop_duplicates()
new_trans = pd.DataFrame()
for dt in trans['date_dt'].drop_duplicates().values:
    temp['date_dt'] = dt
    new_trans = pd.concat([new_trans,temp],axis=0,ignore_index=True)
new_trans = pd.merge(new_trans,trans,how='left',on=new_trans.columns.tolist()).fillna(0)
new_trans = new_trans[trans.columns]
trans = new_trans.copy()
del new_trans
print(flow.columns)
print(trans.columns)

cc_se = pd.Series(trans['o_city_code'].value_counts().index)
dc_se = pd.Series(trans['o_district_code'].value_counts().index)

trans['o_city_code'] = trans['o_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['o_district_code'] = trans['o_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

trans['d_city_code'] = trans['d_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['d_district_code'] = trans['d_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['cnt']
for label in labels:
    train = []
    test = []
    for va in trans[['o_city_code','o_district_code','d_city_code','d_district_code']].drop_duplicates().values:
        occ = va[0]
        odc = va[1]
        dcc = va[2]
        ddc = va[3]
        price = trans[(trans['o_city_code']==occ)&(trans['o_district_code']==odc)&(trans['d_city_code']==dcc)&(trans['d_district_code']==ddc)][label].values.tolist()
        k = len(price)
        diff = 30
        l = [occ,odc,dcc,ddc]
        l.extend(price[k-diff:k])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [occ,odc,dcc,ddc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':0.7, 
        'bagging_freq':1,
        'feature_fraction':0.7, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1,2,3],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=VBE)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='cnt':
        sub = test_x[[0,1,2,3]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['o_city_code'] = sub[0].map(cc_se)
sub['o_district_code'] = sub[1].map(dc_se)
sub['d_city_code'] = sub[2].map(cc_se)
sub['d_district_code'] = sub[3].map(dc_se)

res = pd.read_csv('result/submission_30_3.csv',header=None)  # 1306
res.columns = flow.columns.tolist()
res['flow_out'] = res['district_code'].map(sub.groupby('o_district_code')['cnt'].sum())
res['flow_in'] = res['district_code'].map(sub.groupby('d_district_code')['cnt'].sum())
res.to_csv('result/submission_part_30_3.csv',header=None,index=False)
0.1387

(2480320, 6)
(1407099, 6)
Index(['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in',
       'flow_out'],
      dtype='object')
Index(['date_dt', 'o_city_code', 'o_district_code', 'd_city_code',
       'd_district_code', 'cnt'],
      dtype='object')
(1026648, 34)
(9506, 34)
Training until validation scores don't improve for 200 rounds.
[1000]	train's rmse: 1.26641	evals's rmse: 1.9688
Early stopping, best iteration is:
[1643]	train's rmse: 1.20095	evals's rmse: 1.95827
Training until validation scores don't improve for 200 rounds.
[1000]	train's rmse: 1.30444	evals's rmse: 1.72778
Early stopping, best iteration is:
[1722]	train's rmse: 1.24042	evals's rmse: 1.70896
Training until validation scores don't improve for 200 rounds.
[1000]	train's rmse: 1.29406	evals's rmse: 1.75013
Early stopping, best iteration is:
[1770]	train's rmse: 1.21785	evals's rmse: 1.7377
Training until validation scores don't improve for 200 rounds.
[1000]	train's rmse: 1.30719	evals's rmse: 1.77458
Earl